In [1]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

Copying gs://indaba-data/README.txt...
Copying gs://indaba-data/test/test_mut.pt...
Copying gs://indaba-data/test/test.csv...
Copying gs://indaba-data/test/test_wt.pt...
Copying gs://indaba-data/train/train.csv...
Copying gs://indaba-data/train/train_wt.pt...
Copying gs://indaba-data/train/train_mut.pt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").



In [2]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader


# move to data folder
%cd indaba-data

/content/indaba-data


In [3]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
df = pd.read_csv("train/train.csv")

In [4]:
len(df)

339778

In [5]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Split data into train and validation
wt_emb_train, wt_emb_val, mut_emb_train, mut_emb_val, df_train, df_val = train_test_split(wt_emb, mut_emb, df, test_size=0.21, random_state=42)

# Define the dataset class
class EmbeddingDataset(Dataset):
  def __init__(self, wt_pt, mut_pt, data_df):
    self.pt_wt = wt_pt
    self.pt_mut = mut_pt
    self.df = data_df

  def __len__(self):
    return len(self.pt_wt)

  def __getitem__(self, index):
    if "ddg" in self.df.columns:
      df_out = torch.Tensor([self.df.iloc[index]["ddg"]])
    else:
      df_out = torch.Tensor([self.df.iloc[index]["ID"]])

    return self.pt_wt[index,:], self.pt_mut[index,:], df_out

# Create separate datasets for the training and validation sets
train_dataset = EmbeddingDataset(wt_emb_train, mut_emb_train, df_train.reset_index(drop=True))
val_dataset = EmbeddingDataset(wt_emb_val, mut_emb_val, df_val.reset_index(drop=True))

# Create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [6]:
len(train_dataloader)

8389

In [7]:
len(val_dataloader)


2230

In [8]:
for i, (wt_emb, mut_emb, ddg) in enumerate(train_dataloader):
    print(f'wt_emb shape: {wt_emb.shape}, mut_emb shape: {mut_emb.shape}')
    if i > 5:  # Print shapes for first 5 batches only
        break


wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])


In [11]:
for wt_emb, mut_emb, ddg in train_dataloader:
    print(wt_emb_train.shape, mut_emb_train.shape, ddg.shape)
    break
import torch.nn as nn
import torch.nn.functional as F


torch.Size([268424, 1280]) torch.Size([268424, 1280]) torch.Size([32, 1])


In [12]:
class AttentionLayer(nn.Module):
    def __init__(self, input_dim, attention_dim):
        super(AttentionLayer, self).__init__()
        self.attention_weights = nn.Parameter(torch.randn(input_dim, attention_dim))
        self.fc = nn.Linear(attention_dim, 1)

    def forward(self, x):
        attention_scores = F.softmax(self.fc(x @ self.attention_weights), dim=1)
        return (attention_scores * x)

class MLPWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, attention_dim):
        super(MLPWithAttention, self).__init__()
        self.attention = AttentionLayer(input_dim, attention_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, wt_emb, mut_emb):
        x = torch.cat((wt_emb, mut_emb), dim=1)
        x = self.attention(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return out

# Create the model
input_dim = 1280 * 2  # Multiply by 2 since we're concatenating wt_emb and mut_emb
hidden_dim = 256
output_dim = 1  # ddG value
attention_dim = 512  # Hyperparameter that you may need to tune
model = MLPWithAttention(input_dim, hidden_dim, output_dim, attention_dim)


In [ ]:
# Training loop
import torch.nn.functional as F
num_epochs = 15  # Adjust as needed
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    train_losses = []

    for wt_emb, mut_emb, ddg in train_dataloader:
        optimizer.zero_grad()
        outputs = model(wt_emb, mut_emb)
        loss = criterion(outputs, ddg)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    with torch.no_grad():
        val_losses = []
        for wt_emb, mut_emb, ddg in val_dataloader:
            outputs = model(wt_emb, mut_emb)
            loss = criterion(outputs, ddg)
            val_losses.append(loss.item())
    
    avg_val_loss = sum(val_losses) / len(val_losses)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')


Epoch 1, Train Loss: 0.9231522999555565, Val Loss: 0.8415401782489679
Epoch 2, Train Loss: 0.8382683936539815, Val Loss: 0.8188282968551589
Epoch 3, Train Loss: 0.8246590296788584, Val Loss: 0.8031103128943208
Epoch 4, Train Loss: 0.8174986777977461, Val Loss: 0.8277799490855948
Epoch 5, Train Loss: 0.8142003603754744, Val Loss: 0.8161756578314999
Epoch 6, Train Loss: 0.8104593254610699, Val Loss: 0.8039739552768357
Epoch 7, Train Loss: 0.8071092830335643, Val Loss: 0.8047313353138654
Epoch 8, Train Loss: 0.8047898599645039, Val Loss: 0.8521381614053196


## Prediction & submission

In [ ]:
# load embedding tensors & traing csv
wt_test_emb = torch.load("test/test_wt.pt")
mut_test_emb = torch.load("test/test_mut.pt")
df_test = pd.read_csv("test/test.csv")

In [ ]:
# creating testing dataset and loading the embedding
test_dataset = EmbeddingDataset(wt_test_emb,mut_test_emb,df_test)
# preparing a dataloader for the testing
test_dataloader = torch.utils.data.dataloader.DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=2,
    )

In [ ]:
df_result = pd.DataFrame()
with torch.no_grad():
  for batch_idx, (data_mut,data_wt , target) in tqdm(enumerate(test_dataloader)):
    x1 = data_wt.to(device)
    x2 = data_mut.to(device)
    id = target.to(device)
    # make prediction
    y_pred = model(x1,x2)
    df_result = pd.concat([df_result, pd.DataFrame({"ID":id.squeeze().cpu().numpy().astype(int) , "ddg" : y_pred.squeeze().cpu().numpy()})])

In [ ]:
df_result.to_csv("submission.csv",index=False)